In [9]:
import random
import simpy

In [10]:
RANDOM_SEED = 42
NUM_MACHINES = 2  # Number de maquinas en la lavadora
WASHTIME = 5      # Minutos en limpiar un carro
T_INTER = 7       # Crea un coche cada ~ 7 minutos
SIM_TIME = 20     # Tiempo de simulación en minutos

In [11]:
class Carwash(object):
    #Un túnel de lavado tiene un número limitado de máquinas (` `NUM_MACHINES``) para
    #Limpiar coches en paralelo.
    #Los coches tienen que solicitar una de las máquinas. Cuando consiguieron uno, ellos
    #puede iniciar los procesos de lavado y esperar a que finalice (que
    #toma minutos de `` tiempo de lavado '').
    
    def __init__(self, env, num_machines, washtime):
        self.env = env
        self.machine = simpy.Resource(env, num_machines)
        self.washtime = washtime

    def wash(self, car):
        #Los procesos de lavado. Se necesita un` `coche`` procesa y prueba para limpiarlo.
        yield self.env.timeout(WASHTIME)
        print("La lavadora elimino el %d%% de %s'rro." %
              (random.randint(50, 99), car))


def car(env, name, cw):
    #El proceso del auto (cada auto tiene un` `nombre '') llega al lavado de autos
    #(`` cw '') y solicita una máquina de limpieza.
    #Luego inicia el proceso de lavado, espera a que termine y
    #se va para no volver nunca ...
    print('El %s llega a la lavadora a las %.2f.' % (name, env.now))
    with cw.machine.request() as request:
        yield request

        print('El %s entra a la lavadora a las %.2f.' % (name, env.now))
        yield env.process(cw.wash(name))

        print('El %s sale del tunel de lavado a las %.2f.' % (name, env.now))


def setup(env, num_machines, washtime, t_inter):
    #Crea un lavado de autos, varios autos iniciales y sigue creando autos
    #aprox. cada `` t_inter`` minutos.
    # Creamos la lavadora
    carwash = Carwash(env, num_machines, washtime)

    # Creamos los carros iniciales
    for i in range(4):
        env.process(car(env, 'Carro %d' % i, carwash))

    # Crea más coches mientras se ejecuta la simulación
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        i += 1
        env.process(car(env, 'Carro %d' % i, carwash))

In [12]:
# Configurar e iniciar la simulación

random.seed(RANDOM_SEED)  
env = simpy.Environment()
env.process(setup(env, NUM_MACHINES, WASHTIME, T_INTER))

<Process(setup) object at 0x266bdbf1e88>

In [13]:
print('Lavadora UPS')
env.run(until=SIM_TIME)

Lavadora UPS
El Carro 0 llega a la lavadora a las 0.00.
El Carro 1 llega a la lavadora a las 0.00.
El Carro 2 llega a la lavadora a las 0.00.
El Carro 3 llega a la lavadora a las 0.00.
El Carro 0 entra a la lavadora a las 0.00.
El Carro 1 entra a la lavadora a las 0.00.
El Carro 4 llega a la lavadora a las 5.00.
La lavadora elimino el 97% de Carro 0'rro.
La lavadora elimino el 67% de Carro 1'rro.
El Carro 0 sale del tunel de lavado a las 5.00.
El Carro 1 sale del tunel de lavado a las 5.00.
El Carro 2 entra a la lavadora a las 5.00.
El Carro 3 entra a la lavadora a las 5.00.
El Carro 5 llega a la lavadora a las 10.00.
La lavadora elimino el 64% de Carro 2'rro.
La lavadora elimino el 58% de Carro 3'rro.
El Carro 2 sale del tunel de lavado a las 10.00.
El Carro 3 sale del tunel de lavado a las 10.00.
El Carro 4 entra a la lavadora a las 10.00.
El Carro 5 entra a la lavadora a las 10.00.
La lavadora elimino el 97% de Carro 4'rro.
La lavadora elimino el 56% de Carro 5'rro.
El Carro 4 sale 